In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
pyspark
from pyspark.sql import *
spark = SparkSession \
    .builder \
    .appName("Movie review") \
    .config("spark.some.config.option","some-value") \
    .getOrCreate()

In [2]:
df = spark.read.load("/home/cse587/train1.csv", format='csv', inferSchema="true",header="true", escape = '"', seperator = ",")
label_df = spark.read.load("/home/cse587/mapping.csv", format='csv', inferSchema="true",header="true", escape = '"', seperator = ",")
df_test = spark.read.load("/home/cse587/test.csv", format='csv', inferSchema="true",header="true", escape = '"', seperator = ",")

In [3]:
labels_list = []
for row in label_df.collect():
    labels_list.append(row[1])

In [4]:
# Relate movie_id and movie_name
movie_name = {}
df = df.dropna(subset=['plot', 'genre'])
for row in df.collect():
    if row['movie_id'] != None:
        movie_name[row['movie_id']] = row['movie_name']
        
# Relate movie_id and movie_name
movie_name_test = {}
df_test = df_test.dropna(subset=['plot'])
for row in df_test.collect():
    if row['movie_id'] != None:
        movie_name_test[row['movie_id']] = row['movie_name']

In [5]:
from collections import defaultdict
from pyspark.sql.functions import col, lower, regexp_replace
from pyspark.ml.feature import RegexTokenizer,Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

df = df.select('movie_id', 'movie_name', (lower(regexp_replace('plot',"[^a-zA-Z\\s]","")).alias('plot')), 'genre')
df_test = df_test.select('movie_id', 'movie_name', (lower(regexp_replace('plot',"[^a-zA-Z\\s]","")).alias('plot')))
tokenizer = RegexTokenizer(inputCol='plot', outputCol='plot_token', pattern="\\W")
df = tokenizer.transform(df)
df_test = tokenizer.transform(df_test)                      
remover = StopWordsRemover(inputCol='plot_token', outputCol='plot_stop')
df=remover.transform(df)
df_test=remover.transform(df_test)

In [6]:
from pyspark.ml.feature import HashingTF, IDF

hashingTF = HashingTF(inputCol="plot_stop", outputCol="features_tf",)
featurizedData = hashingTF.transform(df)

idf = IDF(inputCol="features_tf", outputCol="features")
idfModel = idf.fit(featurizedData)
df = idfModel.transform(featurizedData)

df = df.select("movie_id", "movie_name", "plot_stop", "features", "genre")

KeyboardInterrupt: 

In [ ]:
featurizedData = hashingTF.transform(df_test)

#idfModel = idf.fit(featurizedData)
df_test = idfModel.transform(featurizedData)

df_test = df_test.select("movie_id", "movie_name", "features")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

pd_df = df.toPandas()

/usr/local/lib/python3.6/dist-packages/pyspark/sql/dataframe.py:2103: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: VectorUDT
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [0]:
import ast
label_set = [[0]* len(labels_list) for i in range(len(movie_name))]

In [0]:
i = 0
for row in df.collect():
  if row['genre'] != None:
    elements = ast.literal_eval(row['genre'])
    for genre in elements:
      for label in range(len(labels_list)):
        if genre == labels_list[label]:
          label_set[i][label] = 1
  i = i + 1

label_set

[[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [0]:
import numpy as np
cols = []
Y_train = np.array(label_set)
temp = Y_train.transpose()
l = labels_list
for i in range(len(temp)):
    pd_df[str(i+1)] = temp[i]
    cols.append(str(i+1))

In [0]:
df1 = spark.createDataFrame(pd_df)

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Could not convert (262144,[806,12751,14236,20495,31403,42791,47331,48383,54997,67817,91412,104967,197358,224769,242765],[7.572599430986607,6.656308699112452,9.652040972666443,4.220504762332183,4.391944818938604,3.1351085250555997,5.89084085697288,4.696213915065182,9.246575864558277,7.780238795764851,2.2811808061297265,3.4166503350617226,2.384515544838271,3.0658693178117677,5.28894234787808]) with type SparseVector: did not recognize Python value type when inferring an Arrow data type
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


In [0]:
df2 = df1
df3 = df2.select('movie_id','features', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20')
df3.show(truncate=False)

+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.ml.feature import StandardScaler
standardscaler = StandardScaler().setInputCol("features").setOutputCol("scaled_features")
df3 = standardscaler.fit(df3).transform(df3)
df_test = standardscaler.fit(df_test).transform(df_test)
df3= df3.select('movie_id','scaled_features','1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20')

In [0]:
df3.show()
data = df3

+--------+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|movie_id|     scaled_features|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20|
+--------+--------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|23890098|(262144,[806,1275...|  1|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|31186339|(262144,[991,1739...|  1|  0|  0|  0|  1|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  1|  0|  0|
|20663735|(262144,[119,571,...|  1|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|
| 2231378|(262144,[619,1998...|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|  595909|(262144,[1147,191...|  1|  0|  0|  0|  0|  1|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
| 5272176|(262144,[571,4977...|  1|  0|  0|  1|  1|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0| 

In [0]:
import pyspark.sql.functions as f
from pyspark.ml.classification import LogisticRegression,LogisticRegressionModel
from pyspark.sql.functions import when, lit, col
from pyspark.ml.feature import *
from pathlib import Path

In [0]:
for i in range(20):
    file = Path("part2/"+str(i))
    if file.exists():
      print("Loading Model " + i+1)
      load_path = "part2/" + str(i)
      model = LogisticRegressionModel.load("part2/"+str(i))
    else:
      print("Training Model ",i+1,"/ 20")
      lr = LogisticRegression(labelCol = str(20-i),featuresCol="scaled_features")
      model = lr.fit(data)
      path = "part2/"+str(i)
      model.save(path)

    print("Testing Model ",i+1, "/ 20")
    predictions= model.transform(df_test)
    if (i == 0):
        pred1 = predictions.withColumnRenamed("prediction", (str(20-i)))
        pred1 = pred1.select("movie_id", str(20-i))
        df_p = pred1.withColumn(str(20-i), \
              when(pred1[str(20-i)] == 1, 20-i).otherwise(0))
        df_p.show()
    else:
        pred2 = predictions.withColumnRenamed("prediction", (str(20-i)))
        pred2 = pred2.select("movie_id", str(20-i))
        pred2 = pred2.withColumn(str(20-i),when(pred2[str(20-i)] == 1, 20-i).otherwise(0))
        df_p = pred2.join(df_p, "movie_id", "right")
        df_p.show()

Training Model  1 / 20
Testing Model  1 / 20
+--------+---+
|movie_id| 20|
+--------+---+
| 1335380|  0|
|29062594|  0|
| 9252321|  0|
|13455076|  0|
|24165951|  0|
| 1925869|  0|
|10799612|  0|
|28238240|  0|
|17124781|  0|
|28207941|  0|
|19174305|  0|
|18392317|  0|
|34420857|  0|
| 4039635|  0|
| 8034072|  0|
| 4016437|  0|
| 1520023|  0|
|24589422|  0|
|35068740|  0|
|21132951|  0|
+--------+---+
only showing top 20 rows

Training Model  2 / 20
Testing Model  2 / 20
+--------+---+---+
|movie_id| 19| 20|
+--------+---+---+
| 1335380|  0|  0|
|29062594|  0|  0|
| 9252321|  0|  0|
|13455076|  0|  0|
|24165951|  0|  0|
| 1925869|  0|  0|
|10799612|  0|  0|
|28238240|  0|  0|
|17124781|  0|  0|
|28207941|  0|  0|
|19174305|  0|  0|
|18392317| 19|  0|
|34420857|  0|  0|
| 4039635|  0|  0|
| 8034072|  0|  0|
| 4016437|  0|  0|
| 1520023|  0|  0|
|24589422|  0|  0|
|35068740|  0|  0|
|21132951|  0|  0|
+--------+---+---+
only showing top 20 rows

Training Model  3 / 20
Testing Model  3 / 

In [0]:
final = {}
f3 = df_p.drop('movie_id')
to_list = [list(row) for row in f3.collect()]
y_pred = [[int(ele) for ele in sub if ele != 0] for sub in to_list]

In [0]:
import csv
with open('mapping_part1.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['', '0'])
    labels = labels_list
    for i in range (len(labels)):
        writer.writerow((str(i),labels[i]))

In [0]:
l = list(movie_name_test.keys())
with open('sample_part1.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['movie_id', 'predictions'])
    for i in range (len(y_pred)):
        a = ['0'] * 20
        for j in range(len(y_pred[i])):
            a[y_pred[i][j] - 1] = '1'
        temp = ' '.join(a)
        writer.writerow((l[i],temp))